In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini Long Context Window Tutorial - Text Modality
<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/vijaykyr/genai-demos/blob/main/long_context/Gemini_Long_Context_Text.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
</table>

# Overview

Gemini 1.5 Pro supports up to 2 Million input tokens. This is the equivalent of roughly:
- 2000 pages of text
- 11 hours of audio
- 60 minutes of video

This long context window opens up possibilities for prompting on large contexts that previously could only be approximated using pre-processing steps such as Retrieval Augmented Generation (RAG).

In this notebook we will demonstrate long context window (LCW) using the text modality*. We will demonstrate 3 approaches to long context prompting and will compare each of these approaches along the following dimensions of accuracy, latency and cost.

Below is the summary of results, continue on for a detailed analysis of each.

| Trial | Accuracy | Latency | Cost |
|---|---|---|---|
| **Baseline** | 43% (3/7) | 0.4 min | \\$0.004 |
| **LCW - Naive** | 100% (7/7) | 4 min | \\$34.49 |
| **LCW - Batched** | 86% (6/7) | 1 min | \\$4.93 |
| **LCW - Cached** | 100% (7/7) | 4 min | \\$4.68 |


*For example of another modality see the companion video notebook (TODO).

# Setup

## Install Dependencies (If Needed)

The list `packages` contains tuples of package import names and install names. If the import name is not found then the install name is used to install quitely for the current user.

In [2]:
# tuples of (import name, install name)
packages = [
    ('vertexai','google-cloud-aiplatform'),
    ('pandas','pandas'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user

## Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [3]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Authenticate

Authenticate to GCP

In [4]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

# If using local jupyter instance, uncomment and run:
# !gcloud auth login

## Config

Update the below variables to specify the GCP project ID and region you want to use

In [5]:
GCP_PROJECT_ID = "YOUR-PROJECT-ID"
REGION = "us-central1"

## Import Libraries and Initialize Gemini

In [6]:
import datetime

from IPython.display import Markdown
import pandas as pd
from google.cloud.aiplatform_v1 import HarmCategory
import vertexai
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, Part

pd.set_option('display.max_colwidth', None)
vertexai.init(project=GCP_PROJECT_ID, location=REGION)

# Gemini Config
GENERATION_CONFIG=dict(temperature=0)
SAFETY_CONFIG= {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}
model = GenerativeModel(
    "gemini-1.5-pro-001", 
    generation_config=GENERATION_CONFIG,
    safety_settings=SAFETY_CONFIG,
)

# Source Text

To demonstrate Gemini's long context capabilities in the text modality we will do Questions and Answer about the novel [David Copperfield](https://www.gutenberg.org/cache/epub/43111/pg43111) by Charles Dickens. It is ~360K words and ~506K tokens, suffienctly long to evaluate long context capabilities.

The questions were sourced manually from the novel, without any prior knowledge of how they would perform in the various tests. The answers for these questions are roughly evenly distributed throughout the source material (beginning, middle, end) so as to evaluate performance across the full context window.

In [7]:
questions = [
    "What are the first objects that David can recall from his infancy?",
    "At the inn where the mail stops, what is painted on the door?",
    "What name does David's aunt suggest to Mr. Dick they call him by?",
    "Describe the room in which Mr. Copperfield meets Steerforth for breakfast.",
    "After his engagement to Dora, David write Agnes a letter. What is the letter about?",
    "What does David find in the hotel where Mr. Micawber requested him to meet in the middle of the night?",
    "Who are David's final thoughts about in the book?",
]

answers = [
    "His mother with her pretty hair, and Peggotty.",
    "DOLPHIN",
    "Trotwood Copperfield",
    "A snug private apartment, red-curtained and Turkey-carpeted, where the fire burnt bright, and a fine hot breakfast was set forth on a table covered with a clean cloth; and a cheerful miniature of the room, the fire, the breakfast, Steerforth, and all, was shining in the little round mirror over the sideboard.",
    "He writes to Agnes to assure her of his deep love for Agnes and that this was not a passing fancy or hasty decision.",
    "A letter stating that he would appear in the morning at half past nine.",
    "Agnes",
]


# Baseline Behavior

As David Copperfield is a popular classic novel, it is likely Gemini already knows something about it from its internal knowledge. So that we can later measure how adding the novel explicitly as context helps, let's first ask our sample questions without any added context (aka zero-shot).

In [8]:
%%time
prompt_template = "Answer the following question about {context}: {question}"
context = "the book 'David Copperfield' by Charles Dickens"

def evaluate(questions, answers, prompt_template, context, model=model):
    df = pd.DataFrame(columns=['question', 'ground_truth', 'model_response'])
    for i in range(len(questions)):
        prompt = prompt_template.format(context=context, question=questions[i])
        res = model.generate_content(prompt).text
        df.loc[len(df)] = {
            'question':questions[i], 
            'model_response':res, 
            'ground_truth':answers[i],
        }
    return df
df_zeroshot = evaluate(questions, answers, prompt_template, context) 
df_zeroshot

I0000 00:00:1723663937.312999 1919689 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


CPU times: user 102 ms, sys: 51.4 ms, total: 154 ms
Wall time: 25.9 s


,question,ground_truth,model_response
0,What are the first objects that David can recall from his infancy?,"His mother with her pretty hair, and Peggotty.","The first objects David Copperfield remembers from his infancy are **his mother's face and her dress**. He describes her as ""very pretty"" and remembers the ""soft muslin"" of her dress. He also recalls the sensation of her holding him and the feeling of her heart beating against his own. \n"
1,"At the inn where the mail stops, what is painted on the door?",DOLPHIN,"At the inn where the mail stops in ""David Copperfield,"" the door features a **painting of a very large hand in act to discharge an arrow of extraordinary dimensions**. \n"
2,What name does David's aunt suggest to Mr. Dick they call him by?,Trotwood Copperfield,"David's aunt, Betsey Trotwood, suggests that Mr. Dick call David by the name **""Trotwood""**. \n\nShe dislikes the name ""David"" because it was the name of David's father, whom she strongly disapproved of. She believes that calling him ""Trotwood"" will distance him from his father's memory and influence. \n"
3,Describe the room in which Mr. Copperfield meets Steerforth for breakfast.,"A snug private apartment, red-curtained and Turkey-carpeted, where the fire burnt bright, and a fine hot breakfast was set forth on a table covered with a clean cloth; and a cheerful miniature of the room, the fire, the breakfast, Steerforth, and all, was shining in the little round mirror over the sideboard.","While Dickens describes many rooms in detail throughout ""David Copperfield,"" he doesn't specifically describe the room where Mr. Copperfield and Steerforth have breakfast. \n\nIt's important to remember that David Copperfield is the narrator, and the novel is told from his perspective as a young boy. It's possible that David, caught up in the excitement of meeting Steerforth, doesn't pay much attention to the room itself. \n\nHowever, we can make some educated guesses about the setting based on what we know about Mr. Copperfield's social standing and the norms of the time:\n\n* **Location:** The breakfast likely takes place in the Copperfield's home, either in a formal dining room or a more casual parlor used for meals.\n* **Furnishings:** As a respectable family, the room would have been furnished comfortably and appropriately for their social class. This might include a sturdy dining table and chairs, perhaps a sideboard for serving, and potentially decorative elements like a rug or artwork.\n* **Atmosphere:** The atmosphere was likely pleasant and welcoming, as Mr. Copperfield clearly wanted to make a good impression on Steerforth. \n\nUltimately, Dickens leaves the specific details of the room to the reader's imagination, focusing instead on the characters and their interactions. \n"
4,"After his engagement to Dora, David write Agnes a letter. What is the letter about?",He writes to Agnes to assure her of his deep love for Agnes and that this was not a passing fancy or hasty decision.,"In Charles Dickens's ""David Copperfield,"" after David becomes engaged to Dora Spenlow, he does indeed write a letter to Agnes Wickfield. This letter is driven by his guilt and confusion. \n\nHere's a breakdown of the letter's content:\n\n* **Confession of his love for Dora:** David gushes about his love for Dora, describing her beauty and his overwhelming happiness. He portrays their relationship as idyllic and innocent.\n* **Underlying doubts and anxieties:** Despite his enthusiastic portrayal of his love for Dora, David expresses subtle doubts and anxieties about their compatibility. He acknowledges that Agnes might not fully approve of Dora, hinting at his awareness of Dora's lack of practicality and depth compared to Agnes.\n* **Seeking validation and reassurance:** David seeks validation and reassurance from Agnes. He craves her blessing on his engagement, hoping she'll alleviate his unspoken concerns about his choice. He relies on her wisdom and judgment, even as he

### Analysis

- Latency: 25 seconds
- Cost: $0.004
- Accuracy: 3/7

Accuracy is determined manually by comparing the ground truth to the model response. There is some subjectivity in this evaluation, and at the time of this writing Gemini is non-deterministic, so your results may vary slightly. 

In our analysis only 2 answers are unambigiously correct, and another two we  consider close enough to give partial credit, for a total of 1+1+0.5+0.5=3 out of 7, or 43% accuracy.

The cost is negligable.

# Long Context Window

Now let's see if accuracy improves by feeding the entire novel text as context. 

## Download Novel

In [9]:
import requests

url = "https://www.gutenberg.org/ebooks/766.txt.utf-8"
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx and 5xx)
except requests.exceptions.RequestException as e:
    print(f"Error downloading file: {e}")

novel = response.text

## Naive


We will first construct our prompt in a naive way by stuffing the entirety of the novel into the prompt and asking it one question at a time.

In [10]:
%%time
prompt_template = """
Following is the full text of the novel David Copperfield. 
<document>
{context}
</document>
Based on the novel text provided, answer the following: {question}
"""
df_noncache = evaluate(questions, answers, prompt_template, novel) 
df_noncache

CPU times: user 4.07 s, sys: 259 ms, total: 4.33 s
Wall time: 3min 39s


,question,ground_truth,model_response
0,What are the first objects that David can recall from his infancy?,"His mother with her pretty hair, and Peggotty.","The first objects David Copperfield remembers from his infancy are his mother, with her pretty hair and youthful shape, and Peggotty, his nurse, who is described as having ""no shape at all"" and very dark eyes. \n"
1,"At the inn where the mail stops, what is painted on the door?",DOLPHIN,"At the inn where the mail stops, **DOLPHIN** is painted on the door. \n"
2,What name does David's aunt suggest to Mr. Dick they call him by?,Trotwood Copperfield,"David's aunt suggests to Mr. Dick that they call David ""Trotwood"". \n"
3,Describe the room in which Mr. Copperfield meets Steerforth for breakfast.,"A snug private apartment, red-curtained and Turkey-carpeted, where the fire burnt bright, and a fine hot breakfast was set forth on a table covered with a clean cloth; and a cheerful miniature of the room, the fire, the breakfast, Steerforth, and all, was shining in the little round mirror over the sideboard.","The room where David Copperfield meets Steerforth for breakfast at the Golden Cross is described as a ""snug private apartment,"" quite different from the dismal ""little loft over a stable"" David had been initially assigned. Dickens uses these words to describe the room:\n\n* **Red-curtained:** Suggesting warmth and richness.\n* **Turkey-carpeted:** Implying luxury and comfort.\n* **Fine hot breakfast:** Laid out on a table covered with a clean cloth, emphasizing a stark contrast to the previous day's experience.\n* **Cheerful miniature:** A small, round mirror over the sideboard reflects the pleasant scene, further highlighting the cozy atmosphere.\n\nOverall, the room is depicted as a comfortable and inviting space, reflecting Steerforth's own refined taste and his desire to provide a pleasant experience for David. \n"
4,"After his engagement to Dora, David write Agnes a letter. What is the letter about?",He writes to Agnes to assure her of his deep love for Agnes and that this was not a passing fancy or hasty decision.,"After David gets engaged to Dora, he writes Agnes a long letter telling her about his engagement and how blissful he is. He describes how wonderful Dora is and how deeply in love he is with her. He assures Agnes that this is not a passing fancy like his childhood infatuations, but a profound and lasting love. He wants Agnes to know and understand his happiness. \n"
5,What does David find in the hotel where Mr. Micawber requested him to meet in the middle of the night?,A letter stating that he would appear in the morning at half past nine.,"In the middle of the night at the hotel in Canterbury, David finds a **letter** from Mr. Micawber. \n\nThe letter informs him that Mr. Micawber will be appearing in the morning at precisely half past nine. \n"
6,Who are David's final thoughts about in the book?,Agnes,"At the very end of the book, David's final thoughts are about **Agnes**. He reflects on how she has always been his guiding light and support, leading him to be a better person. He imagines her by his side as he closes his life, a constant source of love and inspiration. \n"


### Analysis

- Latency: 4 minutes
- Cost: $34.49
- Accuracy: 7/7

Unsuprisingly latency is much greater since we're increasing our prompt size by 500K tokens. 

Cost is also significantly increased. At the time of this writing (for up to date pricing see https://cloud.google.com/vertex-ai/generative-ai/pricing) Gemini 1.5 Pro costs \\$0.0025/1k input characters. The novel is 1,970,730 characters which amounts to \\$4.93 per invocation. 

However we now have 100% accuracy.

## Batching

One way to save on cost and latency when dealing with long contexts is by batching multiple questions into one prompt. Let's try asking all 7 of our questions at once.

In [11]:
%%time
prompt_template = """
Following is the full text of the novel David Copperfield. 
<document>
{context}
</document>
Based on the novel text provided, answer the following questions:
{question}
"""
prompt = prompt_template.format(context=novel, question=questions)
res = model.generate_content(prompt).text
Markdown(res)

CPU times: user 576 ms, sys: 38 ms, total: 614 ms
Wall time: 42.1 s


Here are the answers to your questions, based on the provided text of *David Copperfield*:

1. **What are the first objects that David can recall from his infancy?**  The first objects David remembers are his mother, with her pretty hair and youthful shape, and Peggotty, with her dark eyes and hard, red cheeks and arms.

2. **At the inn where the mail stops, what is painted on the door?** The door of David's room at the inn has "DOLPHIN" painted on it.

3. **What name does David's aunt suggest to Mr. Dick they call him by?** David's aunt suggests they call him "Trotwood".  This is later shortened to "Trot".

4. **Describe the room in which Mr. Copperfield meets Steerforth for breakfast.** David meets Steerforth for breakfast in a "snug private apartment, red-curtained and Turkey-carpeted, where the fire burnt bright, and a fine hot breakfast was set forth on a table covered with a clean cloth". A miniature of the room is reflected in a small, round mirror over the sideboard.

5. **After his engagement to Dora, David writes Agnes a letter. What is the letter about?** In his letter to Agnes, David tries to convey how happy he is and how much he loves Dora. He stresses that his love for Dora is not a passing fancy and asks Agnes not to see it as similar to the boyish infatuations they used to joke about. He also mentions the sadness in Yarmouth over Emily's disappearance, saying it is a double wound for him due to the circumstances.

6. **What does David find in the hotel where Mr. Micawber requested him to meet in the middle of the night?** David finds a letter from Mr. Micawber stating that there is no hope of the remittance he was expecting and that he is financially ruined. He has settled his debt at the hotel with a note of hand that will not be honored when it is due.

7. **Who are David's final thoughts about in the book?** David's final thoughts are about Agnes. He reflects on how she has always been his guiding light and how his love for her has endured. The book ends with him imagining her by his side as he dies. 


### Analysis

- Latency: 60 seconds
- Cost: $4.93
- Accuracy: 6/7

While we save considerably on latency and cost, it now hallucinates the answer for question 6. Asking several questions in a single prompt, while cost and latency efficient, can sacrifice accuracy. 

You can treat the number of questions per prompt as a sort of hyperparameter, reducing it to prioritize accuracy and increasing it prioritize cost and/or latency. 

## Cached

In cases where we anticipate multiple model invocations about the same long context, instead of passing the whole context in the prompt each time we can take advantage of [context caching](https://cloud.google.com/vertex-ai/generative-ai/docs/context-cache/context-cache-overview).

Caching can be combined with batching to further reduce cost and latency. For example if you have 100 questions, ask in 10 batches of 10 questions. However in this case for the sake of comparison we will forgo batching and ask only one question per prompt.



In [14]:
from vertexai.preview import caching
from vertexai.preview.generative_models import GenerativeModel

system_instruction = """
Here is the full text of the novel David Copperfield. Based on the novel text provided, answer the following questions.
"""

contents = [Part.from_text(novel)]

cached_content = caching.CachedContent.create(
    model_name="gemini-1.5-pro-001",
    system_instruction=system_instruction,
    contents=contents,
    ttl=datetime.timedelta(minutes=10),
)
cached_content = caching.CachedContent(cached_content_name=cached_content.name)
model_cached = GenerativeModel.from_cached_content(
    cached_content=cached_content,
    generation_config=GENERATION_CONFIG,
    safety_settings=SAFETY_CONFIG,
)

I0000 00:00:1723667017.463191 1919689 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723667038.703317 1919689 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723667039.530359 1919689 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [13]:
%%time
df_cache = evaluate(questions, answers, prompt_template, novel, model_cached) 
df_cache

I0000 00:00:1723664256.149148 1919689 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


CPU times: user 4.1 s, sys: 246 ms, total: 4.34 s
Wall time: 3min 39s


,question,ground_truth,model_response
0,What are the first objects that David can recall from his infancy?,"His mother with her pretty hair, and Peggotty.","The first objects David Copperfield remembers from his infancy are his mother, with her pretty hair and youthful shape, and Peggotty, with no shape at all, dark eyes, and hard, red cheeks and arms. \n"
1,"At the inn where the mail stops, what is painted on the door?",DOLPHIN,"At the inn where the mail stops, ""DOLPHIN"" is painted on the door. \n"
2,What name does David's aunt suggest to Mr. Dick they call him by?,Trotwood Copperfield,"David's aunt suggests to Mr. Dick that they call him ""Trotwood"". \n"
3,Describe the room in which Mr. Copperfield meets Steerforth for breakfast.,"A snug private apartment, red-curtained and Turkey-carpeted, where the fire burnt bright, and a fine hot breakfast was set forth on a table covered with a clean cloth; and a cheerful miniature of the room, the fire, the breakfast, Steerforth, and all, was shining in the little round mirror over the sideboard.","The room where David Copperfield meets Steerforth for breakfast is described as a ""snug private apartment."" It is decorated with red curtains and has a Turkey carpet. A bright fire burns in the fireplace, and a hot breakfast is laid out on a table covered with a clean tablecloth. The narrator notes that a miniature reflection of the cozy scene, including the fire, breakfast, and Steerforth himself, is visible in the round mirror over the sideboard. \n"
4,"After his engagement to Dora, David write Agnes a letter. What is the letter about?",He writes to Agnes to assure her of his deep love for Agnes and that this was not a passing fancy or hasty decision.,"After getting engaged to Dora, David writes Agnes a long letter in which he tries to convey his immense happiness and how wonderful Dora is. He stresses that his love for Dora is not a fleeting infatuation like his previous boyish fancies, but a deep and enduring love. He begs Agnes not to see it as anything less than that. \n\nHe also mentions the sad events at Yarmouth regarding Emily's disappearance, stating that it was a double blow for him due to the circumstances surrounding it. However, he doesn't mention Steerforth's involvement, knowing Agnes would be able to deduce the truth without him explicitly stating it. \n"
5,What does David find in the hotel where Mr. Micawber requested him to meet in the middle of the night?,A letter stating that he would appear in the morning at half past nine.,"David finds a letter from Mr. Micawber at the hotel in Canterbury, stating that he will meet David and the others punctually at half past nine in the morning. \n"
6,Who are David's final thoughts about in the book?,Agnes,"At the end of the novel, David's final thoughts are about **Agnes**. He reflects on their journey together through life, surrounded by their children and friends. He acknowledges her unwavering support and guidance throughout his life, even when he was not worthy of it. He recognizes her as the source of his best aspirations and the center of his life. The book concludes with a beautiful and moving tribute to Agnes, highlighting her serenity and the enduring nature of his love for her. \n"


### Analysis

- Latency: 4 minutes
- Cost: \\$4.68 (\\$4.31 query cost + \\$0.37 storage for 10 minutes)
- Accuracy: 7/7


Cached input is almost 10x discounted at \\$0.0003125/1k input characters, plus a storage charge of \\$0.001125/1k characters/hour.

The more often you query, the more the caching approach saves.

There is not a significant latency improvement from caching but accuracy is back at 100%

# Conclusion

| Trial | Accuracy | Latency | Cost |
|---|---|---|---|
| **Baseline** | 43% (3/7) | 0.4 min | \\$0.004 |
| **LCW - Naive** | 100% (7/7) | 4 min | \\$34.49 |
| **LCW - Batched** | 86% (6/7) | 1 min | \\$4.93 |
| **LCW - Cached** | 100% (7/7) | 4 min | \\$4.68 |

We have demonstrated various approaches to long context prompting and compared them across the dimensions of latency, cost and accuracy. 

Whenever using the same long context across multiple prompts, caching is a great option to reduce cost. You can amplify the cost savings and reduce latency by batching, but be careful as batching too many questions will start to negatively impact accuracy.